誤差関数

<img src="loss_function.png">

学習する際は、分割された領域する際は、分割された領域で、
- Pr(obj)・・・バウンディングボックスに物体が含まれる確率(0と1で表された行列　バウンディングボックスの中心がグリットセルに含まれている場合は1、それ以外は0)
- IoU・・・予測されたバウンディングボックスと政界のバウンディングボックスの重なり具合

を掛け算した式「q=Pr(Obj)*IoU」を、バウンディングボックスの信頼度として計算し、誤差関数に加える

物体検出時には、
- P(Class|Obj)・・・その領域がどのクラスに属している確率が高いかという予測と
- q・・・バウンディングボックスの信頼度

を掛け算した式「P(Class|Obj)*q」を使用して、同じ領域に複数のバウンディングボックスが出現しすぎないようにする




- IoUはintersection over unionの略
<img src="https://meideru.com/wp-content/uploads/2017/05/iou_value_1.png">
- 予測された複数のバウンディングボックスのうち、重なっている部分が大きい場合は一つに絞る。逆に重なっている部分が小さい、もしくは全く無いバウンディングボックスの場合は、消去せずに残す。
- Non-Maximum Suppression(https://meideru.com/archives/3538)

<img src="region.png">

- バウンディングボックスの信頼度を予測するものと、どのクラスに分類されているかを予測するネットワークがある。それら２つの意見を合算

- SxS個のグリッドセル
- 最終的な出力は各領域ごとにC個のクラス確率とB個のバウンディングボックスを予測する
- バウンディングボックスは５つの要素からなる
- 最終的な出力はS×S×(B∗5 + C)個のテンソルを出力 それぞれのグリッドセルに、バウンディングボックスに関する位置の情報が含まれている。

<img src="yolo.png">

## 参考文献
github,11月2日参照
https://github.com/hizhangp/yolo_tensorflow/ 

In [1]:
    def loss_layer(self, predicts, labels, scope='loss_layer'):
        with tf.variable_scope(scope):
            predict_classes = tf.reshape(
                predicts[:, :self.boundary1],
                [self.batch_size, self.cell_size, self.cell_size, self.num_class])
            predict_scales = tf.reshape(
                predicts[:, self.boundary1:self.boundary2],
                [self.batch_size, self.cell_size, self.cell_size, self.boxes_per_cell])
            predict_boxes = tf.reshape(
                predicts[:, self.boundary2:],
                [self.batch_size, self.cell_size, self.cell_size, self.boxes_per_cell, 4])

            response = tf.reshape(
                labels[..., 0],
                [self.batch_size, self.cell_size, self.cell_size, 1])
            boxes = tf.reshape(
                labels[..., 1:5],
                [self.batch_size, self.cell_size, self.cell_size, 1, 4])
            boxes = tf.tile(
                boxes, [1, 1, 1, self.boxes_per_cell, 1]) / self.image_size
            classes = labels[..., 5:]

            offset = tf.reshape(
                tf.constant(self.offset, dtype=tf.float32),
                [1, self.cell_size, self.cell_size, self.boxes_per_cell])
            offset = tf.tile(offset, [self.batch_size, 1, 1, 1])
            offset_tran = tf.transpose(offset, (0, 2, 1, 3))
            predict_boxes_tran = tf.stack(
                [(predict_boxes[..., 0] + offset) / self.cell_size,
                 (predict_boxes[..., 1] + offset_tran) / self.cell_size,
                 tf.square(predict_boxes[..., 2]),
                 tf.square(predict_boxes[..., 3])], axis=-1)

            iou_predict_truth = self.calc_iou(predict_boxes_tran, boxes)

            # calculate I tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
            #オブジェクトがあるところの領域をTrue(1)他を0として、マスクを作る。
            object_mask = tf.reduce_max(iou_predict_truth, 3, keep_dims=True)
            object_mask = tf.cast(
                (iou_predict_truth >= object_mask), tf.float32) * response

            # calculate no_I tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
            #オブジェクトがないところの領域を1として、他を0として、マスクを作る。
            noobject_mask = tf.ones_like(
                object_mask, dtype=tf.float32) - object_mask

            boxes_tran = tf.stack(
                [boxes[..., 0] * self.cell_size - offset,
                 boxes[..., 1] * self.cell_size - offset_tran,
                 tf.sqrt(boxes[..., 2]),
                 tf.sqrt(boxes[..., 3])], axis=-1)

            # class_loss
            class_delta = response * (predict_classes - classes)
            class_loss = tf.reduce_mean(
                tf.reduce_sum(tf.square(class_delta), axis=[1, 2, 3]),
                name='class_loss') * self.class_scale

            # object_loss
            #object_mastで、考慮したいlossのみ抽出する。考慮しなくて良いlossは0を掛け算される。
            #predict_scalesは
            #[self.cell_size, self.cell_size, self.boxes_per_cell]の情報を持つ
            #あらかじめトレーニングの際に計算されたiou値と、予測されたiou値の差分が誤差になる。
            object_delta = object_mask * (predict_scales - iou_predict_truth)
            object_loss = tf.reduce_mean(
                tf.reduce_sum(tf.square(object_delta), axis=[1, 2, 3]),
                name='object_loss') * self.object_scale

            # noobject_loss
            #オブジェクトがない領域に0の教師を割り当てて、(予測-0)^2の誤差を計算する。
            #0の引き算の項は明示しなくても良いので、（予測）^2を誤差にしている。
            #オブジェクトがないときは、iou値は0を出力しなければならないのに、0以上の実数値が出た場合は、
            #lossになる
            noobject_delta = noobject_mask * predict_scales
            noobject_loss = tf.reduce_mean(
                tf.reduce_sum(tf.square(noobject_delta), axis=[1, 2, 3]),
                name='noobject_loss') * self.noobject_scale

            # coord_loss
            coord_mask = tf.expand_dims(object_mask, 4)
            boxes_delta = coord_mask * (predict_boxes - boxes_tran)
            coord_loss = tf.reduce_mean(
                tf.reduce_sum(tf.square(boxes_delta), axis=[1, 2, 3, 4]),
                name='coord_loss') * self.coord_scale

            tf.losses.add_loss(class_loss)
            tf.losses.add_loss(object_loss)
            tf.losses.add_loss(noobject_loss)
            tf.losses.add_loss(coord_loss)

## 推論するときのフロー


```python
def process_predicts(predicts):
  p_classes = predicts[0, :, :, 0:20]
  #Cはそれぞれのグリッドセルごとの、バウンディングボックスの中心座標の存在確率
  #それぞれの要素が0~1の行列
  C = predicts[0, :, :, 20:22]
  
  coordinate = predicts[0, :, :, 22:]
  
  #p_classesはそれぞれのグリッドセルが、どのクラスに分類される確率が高いかを表したもの。
  #クラスの数だけレイヤーが存在する。
  p_classes = np.reshape(p_classes, (7, 7, 1, 20))
  C = np.reshape(C, (7, 7, 2, 1))

  P = C * p_classes#バウンディングボックスの中心座標の存在確率*どのクラスに分類されるか

  #print P[5,1, 0, :]

  index = np.argmax(P)#クラスの分類確率とバウンディングボックスの存在確率maxの値のindexを取得

  index = np.unravel_index(index, P.shape)#index取得

  class_num = index[3]#クラスの分類確率とバウンディングボックスの存在確率maxの値が存在するクラスを検出クラスとする

  coordinate = np.reshape(coordinate, (7, 7, 2, 4))

  max_coordinate = coordinate[index[0], index[1], index[2], :]

  xcenter = max_coordinate[0]
  ycenter = max_coordinate[1]
  w = max_coordinate[2]
  h = max_coordinate[3]

  xcenter = (index[1] + xcenter) * (448/7.0)
  ycenter = (index[0] + ycenter) * (448/7.0)

  w = w * 448
  h = h * 448

  xmin = xcenter - w/2.0
  ymin = ycenter - h/2.0

  xmax = xmin + w
  ymax = ymin + h

  return xmin, ymin, xmax, ymax, class_num
 ```

## わからなかったこと
- 大きな物体を検出するときは、バウンディングボックスをどのように描き連結するか、また、なぜそれぞれのグリッドごとにバウンディングボックスを複数用意するのか==>width,heightと中心座標を出力するので、そこから自由にバウンディングボックスを描ける
- 推論時のフロー

## 参考文献
本資料に使われている文献・図は非営利の勉強会などの教育目的に引用されており、引用した図や資料などの著作権は著作者に帰属します。
- https://www.slideshare.net/ssuser07aa33/introduction-to-yolo-detection-model
- https://www.slideshare.net/DeepLearningJP2016/dl-reading-paper20170804pdf
- https://arxiv.org/abs/1506.02640 (You Only Look Once: Unified, Real-Time Object Detection,参照日　2018/11/3)
- https://www.slideshare.net/ssuser07aa33/introduction-to-yolo-detection-model
- https://github.com/llSourcell/YOLO_Object_Detection
- https://github.com/hizhangp/yolo_tensorflow/
- https://github.com/nilboy/tensorflow-yolo/